In [1]:
import numpy as np
import pandas as pd
import os
import glob
from helper import get_data_splits, id_key, get_model

In [2]:
# Below identifiers correspond to evaluation protocol 1, 2 and 3 respectively
# identifiers - "TouchPose_crossSession", "TouchPose_crossSubjects", "TouchPose_crossGestures"
identifier = "TouchPose_crossGestures"
data_path = "../data/"
model_path = "../models/"

In [3]:
files = glob.glob(data_path+'*.pkl')
df = pd.concat([pd.read_pickle(fp) for fp in files], ignore_index=True)

In [4]:
control = id_key[identifier]
df_group = df.groupby([control])["cap_img","depth_img"]
g_id = list(df_group.groups.keys())

/usr/local/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [5]:
depth_err = []
for i in range(len(g_id)):
    test_set = [g_id[i]]
    X_test_cap, Y_test_depth= get_data_splits(df_group,test_set)
    model = get_model(identifier, model_path, test_set)
    Y_out_depth = model.predict(X_test_cap, batch_size=128)
    depth_error = np.mean(abs(Y_test_depth - Y_out_depth[:,:,:,0])) * 255
    depth_err.append(depth_error)
    print("Accuracy", g_id[i],depth_error)

Accuracy 65.0 30.418463475264414
Accuracy 66.0 24.826761955218785
Accuracy 67.0 21.052762648514722
Accuracy 68.0 23.050934061623348
Accuracy 69.0 25.860948980721368
Accuracy 70.0 18.41753416478953
Accuracy 71.0 16.926196823454315
Accuracy 72.0 19.044581848936193
Accuracy 73.0 14.32355895091892
Accuracy 74.0 16.59890028697684
Accuracy 75.0 26.522050507939174
Accuracy 76.0 37.607740314526104
Accuracy 77.0 35.52240715930311
Accuracy 78.0 30.61358710123129
Accuracy 79.0 33.17285629285531


In [6]:
print(identifier)
print("\t Depth Error",np.mean(depth_err),"SD",np.std(depth_err))

TouchPose_crossGestures
	 Depth Error 24.930618971484893 SD 7.0629837953320145
